<a href="https://colab.research.google.com/github/serhatataman/ProjectAmbitionColab/blob/main/Outsmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outsmart



## Setup

### Enable GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr 25 01:35:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Mount to the Google Drive

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Using Google CoLab")
except:
    print("Not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Using Google CoLab



### Import NVIDIA stylegan2 ada

In [ ]:
%cd "/content/drive/MyDrive/Outsmart"
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
%cd "/content/drive/MyDrive/Outsmart/stylegan2-ada-pytorch"
!git config --global user.name "serhatataman"
!git config --global user.email "serhatataman13@hotmail.com"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/MyDrive/Outsmart/stylegan2-ada-pytorch


### Download from webscrap of WikiArt

In [ ]:
import urllib
import re
from bs4 import BeautifulSoup
import time

file_path = "/content/drive/MyDrive/Outsmart/raw_data"
base_url = "https://www.wikiart.org"

# iterate through all artists by last name alphabetically
for c in range(ord('a'), ord('z') + 1):
    char = chr(c)
    artist_list_url = base_url + '/en/Alphabet/' + char + '/text-list'

    genre_soup = BeautifulSoup(urllib.request.urlopen(artist_list_url), "lxml")
    artist_list_main = genre_soup.find("main")
    lis = artist_list_main.find_all("li")

    # for each list element
    for li in lis:

        # get the date range
        for line in li.text.splitlines():
            # if line.startswith(",") and "-" in line:
            #     parts = line.split('-')
            #     if len(parts) == 2:
            #         born = int(re.sub("[^0-9]", "", parts[0]))
            #         died = int(re.sub("[^0-9]", "", parts[1]))

            # look for artists who may have created work that could in public domain
            # if born > 1850 and died > 0 and (born < 1900 or died < 1950):

            link = li.find("a")
            if link is None:
                continue

            artist = link.attrs["href"]

            # get the artist's main page
            artist_url = base_url + artist
            artist_soup = BeautifulSoup(urllib.request.urlopen(artist_url), "lxml")

            # only look for artists with the word abstract on their main page
            if "Abstract" in artist_soup.text or "abstract" in artist_soup.text or "Avant-garde" \
                    in artist_soup.text or "avant-garde" in artist_soup.text:
                print('Artist: ' + artist)

                # get the artist's web page for the artwork
                url = base_url + artist + '/all-works/text-list'

                try:
                    artist_work_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")
                except:
                    print("Error retrieving artist's work list. Url was: " + url)
                    continue

                # get the main section
                artist_main = artist_work_soup.find("main")
                image_count = 0
                artist_name = artist.split("/")[2]

                # get the list of artwork
                lis = artist_main.find_all("li")

                # for each list element
                for li in lis:
                    link = li.find("a")

                    if link != None:
                        painting = link.attrs["href"]

                        # get the painting
                        url = base_url + painting
                        # print('Painting base url: ' + url)

                        try:
                            painting_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

                        except:
                            print("error retrieving page")
                            continue

                        # check the copyright
                        if "Public domain" in painting_soup.text:

                            # check the genre
                            genre = painting_soup.find("span", {"itemprop": "genre"})
                            if genre != None and genre.text == "abstract":

                                # get the url
                                og_image = painting_soup.find("meta", {"property": "og:image"})
                                image_url = og_image["content"].split("!")[0]  # ignore the !Large.jpg at the end

                                save_path = file_path + "/" + artist_name + "_" + str(image_count) + ".jpg"

                                # download the file
                                try:
                                    print(f"Downloading {image_url} to {save_path}")
                                    time.sleep(0.2)  # try not to get a 403
                                    urllib.request.urlretrieve(image_url, save_path)
                                    image_count = image_count + 1
                                except Exception as e:
                                    print("Failed downloading " + image_url, e)


### Download zip of art pieces

In [ ]:
import requests

zip_file_url = 'http://web.fsktm.um.edu.my/~cschan/source/ICIP2017/wikiart.zip'

# NOTE the stream=True parameter below
with requests.get(zip_file_url, stream=True) as response:
  response.raise_for_status()
  handle = open('/content/drive/MyDrive/ProjectAmbition/downloads/data.zip', "wb")
  for chunk in response.iter_content(chunk_size=8192): 
      # If you have chunk encoded response uncomment if
      # and set chunk_size parameter to None.
      #if chunk: 
      handle.write(chunk)

  handle.close()

# Note that the number of bytes returned using iter_content is not exactly the chunk_size;
# it's expected to be a random number that is often far bigger, and is expected to be different in every iteration.
# See body-content-workflow and Response.iter_content for further reference.

print('Download completed...')

Download completed...


Unzip downloaded files

In [ ]:
import zipfile

zip_filename = "/content/drive/MyDrive/ProjectAmbition/downloads/data.zip"
directory_to_extract_to = "/content/drive/MyDrive/ProjectAmbition/downloads/"

with zipfile.ZipFile(zip_filename, "r") as zip_ref:
    for name in zip_ref.namelist():
        try:
            zip_ref.extract(name, directory_to_extract_to)
        except (Exception, zipfile.BadZipFile) as e:
            print("A file is corrupted. Filename: " + str(name))
            print(e)

print("Unzip successful...")

A file is corrupted. Filename: wikiart/Baroque/rembrandt_woman-standing-with-raised-hands.jpg
Bad CRC-32 for file 'wikiart/Baroque/rembrandt_woman-standing-with-raised-hands.jpg'
A file is corrupted. Filename: wikiart/Post_Impressionism/vincent-van-gogh_l-arlesienne-portrait-of-madame-ginoux-1890.jpg
Error -3 while decompressing data: invalid block type
Unzip successful...


## Move Images

Move images from different sources to a single file

In [ ]:
import shutil
import os


target_dir = '/content/drive/MyDrive/ProjectAmbition/raw_data/'

modern = '/content/drive/MyDrive/ProjectAmbition/downloads/wikiart/Art_Nouveau_Modern/'
abstract = '/content/drive/MyDrive/ProjectAmbition/downloads/wikiart/Abstract_Expressionism/'
fauvism = '/content/drive/MyDrive/ProjectAmbition/downloads/wikiart/Fauvism/'
impressionism = '/content/drive/MyDrive/ProjectAmbition/downloads/wikiart/Impressionism/'

print('Art Nouveau has: ' + str(len(os.listdir(modern))))
print('Abstract has: ' + str(len(os.listdir(abstract))))
print('Fauvism has: ' + str(len(os.listdir(fauvism))))
print('Impressionism has: ' + str(len(os.listdir(impressionism))))

# Exception is handled in case there are duplicated images

print("Moving Art Nouveau files...")    
file_names = os.listdir(modern)
for file_name in file_names:
  try:
    shutil.move(os.path.join(modern, file_name), target_dir)
  except Exception as e: 
    print(e)
    continue

print("Moving Abstract files...")    
file_names = os.listdir(abstract)
for file_name in file_names:
  try:
    shutil.move(os.path.join(abstract, file_name), target_dir)
  except Exception as e: 
    print(e)
    continue

print("Moving Fauvism files...")    
file_names = os.listdir(fauvism)
for file_name in file_names:
  try:
    shutil.move(os.path.join(fauvism, file_name), target_dir)
  except Exception as e: 
    print(e)
    continue

print("Moving Impressionism files...")    
file_names = os.listdir(impressionism)
for file_name in file_names:
  try:
    shutil.move(os.path.join(impressionism, file_name), target_dir)
  except Exception as e: 
    print(e)
    continue

print("Moving files is successful!")


In [ ]:
target_dir = '/content/drive/MyDrive/ProjectAmbition/raw_data/'

print('Target direcotry has: ' + str(len(os.listdir(target_dir))))

Target direcotry has: 20903


## Cleanup images

See what dataset_tool.py can do

In [ ]:
cmd = f"/usr/bin/python3 /content/drive/MyDrive/ProjectAmbition/stylegan3/dataset_tool.py --help"
!{cmd}

### Remove frames

Remove frames if there are any. This process overrides the picture with frames removed.

In [ ]:
import os
import numpy as np
from PIL import Image
from scipy.spatial import distance

# This python script removes pictures' frames if there are any

from_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'
to_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'


def find_left():
    left = 0
    for i in range(0, w_pad):
        r_stdev = np.std(np_img[h_pad:-h_pad, i:i + 1, 0:1])
        g_stdev = np.std(np_img[h_pad:-h_pad, i:i + 1, 1:2])
        b_stdev = np.std(np_img[h_pad:-h_pad, i:i + 1, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[h_pad:-h_pad, i:i + 1, 0:1])
        g_med = np.median(np_img[h_pad:-h_pad, i:i + 1, 1:2])
        b_med = np.median(np_img[h_pad:-h_pad, i:i + 1, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        left = left + 1
    return left


def find_top():
    top = 0
    for i in range(0, h_pad):
        r_stdev = np.std(np_img[i:i + 1, w_pad:-w_pad, 0:1])
        g_stdev = np.std(np_img[i:i + 1, w_pad:-w_pad, 1:2])
        b_stdev = np.std(np_img[i:i + 1, w_pad:-w_pad, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[i:i + 1, w_pad:-w_pad, 0:1])
        g_med = np.median(np_img[i:i + 1, w_pad:-w_pad, 1:2])
        b_med = np.median(np_img[i:i + 1, w_pad:-w_pad, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        top = top + 1
    return top


def find_right(right):
    right = w
    for i in range(0, w_pad):
        r_stdev = np.std(np_img[h_pad:-h_pad, w - i - 1:w - i, 0:1])
        g_stdev = np.std(np_img[h_pad:-h_pad, w - i - 1:w - i, 1:2])
        b_stdev = np.std(np_img[h_pad:-h_pad, w - i - 1:w - i, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[h_pad:-h_pad, w - i - 1:w - i, 0:1])
        g_med = np.median(np_img[h_pad:-h_pad, w - i - 1:w - i, 1:2])
        b_med = np.median(np_img[h_pad:-h_pad, w - i - 1:w - i, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        right = right - 1
    return right


def find_bottom(bottom):
    for i in range(0, h_pad):
        r_stdev = np.std(np_img[h - i - 1:h - i, w_pad:-w_pad, 0:1])
        g_stdev = np.std(np_img[h - i - 1:h - i, w_pad:-w_pad, 1:2])
        b_stdev = np.std(np_img[h - i - 1:h - i, w_pad:-w_pad, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[h - i - 1:h - i, w_pad:-w_pad, 0:1])
        g_med = np.median(np_img[h - i - 1:h - i, w_pad:-w_pad, 1:2])
        b_med = np.median(np_img[h - i - 1:h - i, w_pad:-w_pad, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        bottom = bottom - 1
    return bottom


for file in os.listdir(from_path):
    path = os.path.join(from_path, file)
    img = Image.open(path)
    file_name, file_extension = os.path.splitext(path)
    print('Removing frames for image:', file_name + file_extension)

    np_img = np.asarray(img)
    # print("shape = " + str(np_img.shape))

    thresh1 = 15000
    thresh2 = 30
    w = img.width
    h = img.height
    pad = 30
    w_pad = w // pad
    h_pad = h // pad

    r_global_med = np.median(np_img[h_pad:-h_pad, w_pad:-w_pad, 0:1])
    g_global_med = np.median(np_img[h_pad:-h_pad, w_pad:-w_pad, 1:2])
    b_global_med = np.median(np_img[h_pad:-h_pad, w_pad:-w_pad, 2:3])

    left = find_left()
    top = find_top()
    right = find_right(w)
    bottom = find_bottom(h)

    # print("left = " + str(left) + ", top = " + str(top) +
      #    ", right = " + str(right) + ", bottom = " + str(bottom) + "\n")

    # img.save(to_path + file)  # save the original
    cropped_img = img.crop((left, top, right, bottom))
    cropped_img.save(file_name + file_extension)  # and the cropped version


print("All images have been cleaned from frames!")


### Resize images

Resize images to 1024x1024 and override them.

In [ ]:
import os
from PIL import Image

image_source_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'
image_target_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'

# We are using Pillow to resize all images to our desired size

for filename in os.listdir(image_source_path):
    path = os.path.join(image_source_path, filename)
    image = Image.open(path).resize((1024, 1024), Image.ANTIALIAS)

    resized_image = image.save(image_target_path + filename)
    print(f"{filename} image is resized...")

print("Resizing of images is successful!")

## Convert images

Convert raw data images to dataset by using StyleGAN3's built-in dataset_tool.py

In [ ]:

for i in range(1, 10):
    print(i)

1
2
3
4
5
6
7
8
9


In [ ]:
from distutils.dir_util import copy_tree

for i in range(18, 21):
  dest = 'dataset/000' + str(i)
  copy_tree('/content/drive/MyDrive/ProjectAmbition/' + str(dest), '/content/drive/MyDrive/Outsmart/' + str(dest))

In [ ]:
!python /content/drive/MyDrive/Outsmart/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/Outsmart/raw_data/ --dest /content/drive/MyDrive/Outsmart/dataset/ --width=1024 --height=1024

The following command can be used to clear out the newly created dataset. If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [ ]:
# !rm -R /content/drive/MyDrive/ProjectAmbition/dataset/*

## Training

### Initial training

In [ ]:
cmd = f"/usr/bin/python3 /content/drive/MyDrive/ProjectAmbition/stylegan3/train.py --help"
!{cmd}

Install required libraries. Must be installed

In [ ]:
%tensorflow_version 1.x
!pip install ninja
#!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1
#!pip install transformers==4.8.0

     |████████████████████████████████| 1.9 MB 16.2 MB/s 
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0+cu113
    Uninstalling torchaudio-0.11.0+cu113:
      Successfully uninstalled torchaudio-0.11.0+cu113


In [ ]:
import os

# !export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

# Modify these to suit your needs
OUTPUT = "/content/drive/MyDrive/Outsmart/output"
DATA = "/content/drive/MyDrive/Outsmart/dataset"
# Snap: How often should the model generate samples and a .pkl file
SNAP = 5
# Mirrored: Should the images be mirrored left to right?
MIRRORED = 1


# Build the command and run it
cmd = f"/usr/bin/python3 /content/drive/MyDrive/Outsmart/stylegan2-ada-pytorch/train.py --cfg=stylegan2 --gpus=1 --gamma=8 --snap {SNAP} --outdir {OUTPUT} --data {DATA} --mirror={MIRRORED} --aug=ada"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 5,
  "network_snapshot_ticks": 5,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/Outsmart/dataset",
    "use_labels": false,
    "max_size": 20903,
    "xflip": true,
    "resolution": 1024
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "ch

### Resume training

Install necessary files first, then resume training

**Note:** `%tensorflow_version` 1.x is needed for this flow to work!

In [ ]:
%tensorflow_version 1.x
!pip install ninja
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1 torchtext==0.9.1

import os

# Modify these to suit your needs
OUTPUT = "/content/drive/MyDrive/Outsmart/output"
FOLDER = "00062-dataset-mirror-stylegan2-gamma8-ada-resumecustom"
NETWORK = "network-snapshot-000120.pkl"
RESUME = os.path.join(OUTPUT, FOLDER, NETWORK)
DATA = "/content/drive/MyDrive/Outsmart/dataset"
# Snap: How often should the model generate samples and a .pkl file
SNAP = 10
# Mirrored: Should the images be mirrored left to right?
MIRRORED = 1

# Build the command and run it
cmd = f"/usr/bin/python3 /content/drive/MyDrive/Outsmart/stylegan2-ada-pytorch/train.py --cfg=stylegan2 --gpus=1 --gamma=8 --snap {SNAP} --resume {RESUME} --outdir {OUTPUT} --data {DATA} --mirror={MIRRORED} --aug=ada"
!{cmd}

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 7.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 17.4 MB 43.7 MB/s 
     |████████████████████████████████| 1.9 MB 87.1 MB/s 
     |████████████████████████████████| 7.1 MB 40.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu113
    Uni

## Generate images

In [ ]:
%tensorflow_version 1.x
!pip install ninja
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1 torchtext==0.9.1

import os
import random


# Modify these to suit your needs
NUMBER_OF_IMAGES_TO_PRODUCE = 100
RESULTS = "/content/drive/MyDrive/Outsmart/results"
OUTPUT = "/content/drive/MyDrive/Outsmart/output"
FOLDER = "00049-dataset-mirror-stylegan2-gamma8-ada-resumecustom"
NETWORK = "network-snapshot-000120.pkl"
MODEL = os.path.join(OUTPUT, FOLDER, NETWORK)

randomlist = []
for i in range(NUMBER_OF_IMAGES_TO_PRODUCE):
  randomlist.append(random.randint(1,10000))

random_numbers_string = ",".join([str(i) for i in randomlist])

cmd = f"/usr/bin/python3 /content/drive/MyDrive/Outsmart/stylegan2-ada-pytorch/generate.py --outdir={RESULTS} --trunc=1 --seeds={random_numbers_string} --network={MODEL}"
!{cmd}

print("All images are successfully generated!")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Loading networks from "/content/drive/MyDrive/Outsmart/output/00049-dataset-mirror-stylegan2-gamma8-ada-resumecustom/network-snapshot-000120.pkl"...
Generating image for seed 9391 (0/100) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 978 (1/100) ...
Generating image for seed 5059 (2/100) ...
Generating image for seed 8054 (3/100) ...
Generating image for seed 2919 (4/100) ...
Generating image for seed 182 (5/100) ...
Generating image for seed 6524 (6/100) ...
Generating image for seed 142 (7/100) ...
Generating image for seed 955 (8/100) ...
Generating image for seed 5178 (9/100) ...
Generating image for seed 7623 (10/100) ...
Generating image for seed 4896 (11/100) ...
Generating image for seed 182 (1